In [ ]:
import requests
import time
import json
import pandas as pd
import requests
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
filename="partidas-1.csv"
equipo1 = []
equipo0 = []
matchID = []
duration = []
partidas=[]
victoria = []

def limpiar_datos(dartaframe):
    df = dartaframe
    df['equipo0'] = df['equipo0'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df['equipo1'] = df['equipo1'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_limpio = df[(df['equipo0'].apply(len) == 5) & (df['equipo1'].apply(len) == 5)]
    return df_limpio

def get_public_matches():
    url = "https://api.opendota.com/api/publicMatches"
    response = requests.get(url)
    return response.json()

def get_match_details(match_id):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    return response.json()

def analizar_heroes(json_data):
    for partida in json_data:
        heroes_equipo0 = []
        heroes_equipo1 = []
        for item in partida:
            if item.get('is_pick') and item.get('team') == 1:
                heroes_equipo1.append(item.get('hero_id'))
            elif item.get('is_pick') and item.get('team') == 0:
                heroes_equipo0.append(item.get('hero_id'))
        
        equipo0.append(heroes_equipo0)
        equipo1.append(heroes_equipo1)
    
    return equipo1, equipo0

def extract_match_info(match_details):    
    picks_bans = match_details.get('picks_bans', [])
    duration = match_details.get('duration')
    # informa 1 si la victoria es del equipo radiant, 0 si la victoria es del equipo dire
    victoria = 1 if match_details.get('radiant_win') == True else 0
    return duration, picks_bans, victoria
    
def unir_datos( equipo0, equipo1, victoria):
    # Crear una lista de diccionarios con los datos
    data = []
    for i in range(len(matchID)):
        data.append({
            "equipo0": equipo0[i],
            "equipo1": equipo1[i],
            "victoria": victoria[i],
        })
    return data
    

    # Crear un DataFrame de pandas
   # df = pd.DataFrame(data)

    # Exportar a un archivo CSV
    #df.to_csv( path, filename, index=False)
# Función principal
def main():
    public_matches = get_public_matches()

    

    for match in public_matches[:100]:  # Limitamos a las primeras 10 partidas para evitar demasiadas solicitudes
        match_id = match['match_id']
        match_details = get_match_details(match_id)
        match_duration, picks_bans, winner = extract_match_info(match_details)
        
        heroes_equipo1, heroes_equipo0 = analizar_heroes([picks_bans])
        matchID.append(match_id)
        duration.append(match_duration)
        victoria.append(winner)

    data = unir_datos( equipo0, equipo1, victoria)

    # Crear un DataFrame de pandas
    print(data)
    df = pd.DataFrame(data)
    dataLimpio = limpiar_datos(df)
    print(df)
    dataLimpio.to_csv( filename, index=False)

    return df


if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import os

ruta_archivos = 'C:/Users/Jules/'  # Reemplaza con la ruta correcta
archivos_csv = [f for f in os.listdir(ruta_archivos) if f.endswith('.csv')]
# Leer y concatenar los archivos CSV
dataframes = []
for archivo in archivos_csv:
    df = pd.read_csv(os.path.join(ruta_archivos, archivo))
    dataframes.append(df)

df_concatenado = pd.concat(dataframes, ignore_index=True)

df_concatenado

df_concatenado.to_csv( "datosconv.csv", index=False)

print(df_concatenado)

In [ ]:
ruta_archivos = 'C:/Users/Jules/'  # Reemplaza con la ruta correcta
archivos_csv = [f for f in os.listdir(ruta_archivos) if f.endswith('.csv')]
# Leer y concatenar los archivos CSV
dataframes = []
for archivo in archivos_csv:
    df = pd.read_csv(os.path.join(ruta_archivos, archivo))
    dataframes.append(df)

df_concatenado = pd.concat(dataframes, ignore_index=True)

df_concatenado

df_concatenado.to_csv( "datosconv3.csv", index=False)

print(df_concatenado)